In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
index_ent=pd.read_csv("../input/all-csv/index_df (1).csv")

In [3]:
index_ent.head()

,index_text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,text
0,"NAME INDEX TO VOLUME 7 OF LETTERS SENT , JULY ...","['Abbott', 'Lyman', 'Adams', 'John Q', 'Allen'...",[],[],"NAME INDEX TO VOLUME 7 OF LETTERS SENT , JULY ..."
1,"Boyd , J. F. Col. 19 , 41 Boyden , James 116 B...","['Boyd', 'J. F. Col.', 'Boyden', 'James', 'Bra...",[],['Henry Corp'],"Boyd , J. F. Col. 19 , 41 Boyden , James 116 B..."
2,"Chur , J. F. Capt . 139 Cilley , Clinton A. Ma...","['Chur', 'J. F.', 'Cilley', 'Clinton A. Maj', ...",[],['Citizens'],"Chur , J. F. Capt . 139 Cilley , Clinton A. Ma..."
3,"Elijah 176 Eliza 172 Emery , Charles Capt . 3 ...","['Elijah', 'Eliza', 'Emery', 'Charles', 'Edwin...",[],[],"Elijah 176 Eliza 172 Emery , Charles Capt . 3 ..."
4,"Hale , Capt . 51 Hambrich , P.R . Capt . 9 Han...","['Hale', 'Hambrich', 'Hand', 'Harkness', 'E.J'...",['N.C'],[],"Hale , Capt . 51 Hambrich , P.R . Capt . 9 Han..."


In [4]:
index_text=index_ent["index_text"]
index_names=index_ent["PERSON-nerenglarge"]
index_loc=index_ent["LOC-nerenglarge"]
index_org=index_ent["ORG-nerenglarge"]

In [5]:
index_text[150]

'1865 . London Manger 14 . London Sarah E 16 . Lippett John E 18 . Lippett W.H 18 . 23 . 33 . Laslo J.G 19 . Lehman R.F 20 . 44 . Lecraft Mary 21 . Lazarus Julia 22 . Lippett W. 23 . 33 . Lecraft Benjamin 36 . 59 . Lewis K.H 30 . Lewis W.F 30 . Levy T.J. 31 . Lane H.B 37 . Leach J A 40 . 45 . Leavy J.P 50 . Lord Fredk 52 . Littlejohn W.H 53 . Leigh J.W 66 . Low T.B 66 .'

In [6]:
index_names[150]

"['London Sarah E', 'Lippett John E', 'Lippett W.H', 'Lehman R.F', 'Lazarus Julia', 'Lippett W', 'Lecraft Benjamin', 'Lewis K.H', 'Lewis W.F', 'Levy T.J', 'Lane H.B', 'Leach J A', 'Leavy J.P', 'Lord Fredk', 'Littlejohn W.H', 'Leigh J.W', 'Low T.B']"

In [7]:
index_text[1500]

'298 WHEN RECEIVED . NAME OF WRITER . DATE AND PURPORT OF LETTER . 299 WHEN RECEIVED . NAME OF WRITER . DATE AND PURPORT OF LETTER . 1.3.70 . 1 Missions , Presby Home . Rev . E. F. Hatfield Secretary per . W. T. M. New York City. , Dec. 30th 1869 . States reply , Mr. Swan , teacher due three months salary , 60 withheld case regard purchase of books investigated . 1.8.70 . 2 Moss , R. C. Teacher . Bladensburg , Md . Jany 6th 1870 . States 81 70100 still due School house lot : names of trustees Darius Bowman , Uriah Wallace , Davis Greenleaf , Cyrus Johnston . 1 , 13 , 70 3 Morris Susan , Teacher . Poolsville , Montgomery Co. Md . January 3 , 1870 . Requests check amount due sent Mr. Adams . 1 , 17 , 70 4 Miller Lewis E. Teacher Lewistown , Frederick Co. Md . Pock Mark Jany 15th 70 , States opening of school Lewistown , forwards compliance instructions Baltimore Association . Will report , instructed report made . 1 , 18 , 70 , 5 Moses Rebecca , Teacher Point of Rocks , Fdk . Co. Md . P.

In [8]:
index_names[1500]

"['E. F. Hatfield', 'Swan', 'Darius Bowman', 'Uriah Wallace', 'Davis Greenleaf', 'Cyrus Johnston', 'Morris Susan', 'Miller Lewis E.', 'Pock Mark', 'Moses Rebecca', 'Moss R', 'C.']"

In [9]:
index_text[5000]

'Jenny 90 . Johnson Richard 10 Jones Andrew 12 Johnston Thos . P. A.N.U . 16 , 20 , 75 Jordan Alice 37 , 53 Jordan Vina 51 , 54 , 55 Johnston Tho.P . A.N.U . 65 , 74 95 , 169 , 199 , 200 , 204 , 205 , 281 Johnston Hugh 107 , 146 Jane 129 Jennings Mr 184 , 185 Johnston David 219 Johnson Leah 228 Jeremia Parthe 253'

In [10]:
index_names[5000]

"['Johnson Richard', 'Jones Andrew', 'Johnston Thos', 'Jordan Alice', 'Jordan Vina', 'Johnston Hugh', 'Jane', 'Jennings Mr', 'Johnston David', 'Johnson Leah', 'Jeremia Parthe']"

In [11]:
index_names

0       ['Abbott', 'Lyman', 'Adams', 'John Q', 'Allen'...
1       ['Boyd', 'J. F. Col.', 'Boyden', 'James', 'Bra...
2       ['Chur', 'J. F.', 'Cilley', 'Clinton A. Maj', ...
3       ['Elijah', 'Eliza', 'Emery', 'Charles', 'Edwin...
4       ['Hale', 'Hambrich', 'Hand', 'Harkness', 'E.J'...
                              ...                        
5417                                                   []
5418    ['Wm . F. Richardson', 'Hattie M. Richardson',...
5419    ['Hattie L. Newcomb', 'Mutch Opelika', 'Thomas...
5420    ['Mutch Opelika', 'Thomas H. Whitly', 'Emma A ...
5421    ['James Nichols Stevenson', 'O . M. Waring', '...
Name: PERSON-nerenglarge, Length: 5422, dtype: object

In [12]:
PL=[]
empty=0
for i in index_names:
    for j in i:
        PL.append(j)
for i in index_names:
    if i==[]:
        empty=empty+1
len(PL), empty

(1298703, 0)

# Names from Index are Perfect 

****

# Order ontonotes

In [13]:
order_ent=pd.read_csv("../input/all-csv/to_compare_result_order.csv")

In [14]:
order_ent.head()

,order_text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,postDate-ontonotes,standard_date-ontonotes,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Other_name-nerenglish
0,R G. R.R . THIS ORDER NOT TRANSFERABLE . U.S. ...,"['E . Cranford', 'Lewis B. Parsons', 'Major Kn...","['Raleigh N.C', 'Warrenton N.C', 'New York']",['Samuel Gale Co .'],"['Aug 6 1866', 'Aug 1866']",[],['67487'],['6-Aug-1866'],['6-August-1866'],"['Cranford', 'Lewis B. Parsons', 'Gaston R.R']","['U.S.', 'Raleigh N.C', 'Warrenton N.C', 'Rale...","['Samuel Gale Co', 'Regiment Freedman', 'Rail ...",[],[],"['Ral', 'Gaston R.R', 'Raleigh N.C.', 'Broadwa...",['Cranford'],[]
1,"Certified Copy of Order , Extract , within tra...",['E. Crawford'],['Raleigh N.C'],[],['Aug 6 1866'],[],[],['6-Aug-1866'],['6-August-1866'],"['Robinson Clinton', 'E. Crawford']",['Raleigh N.C'],[],[],[],[],[],[]
2,"Bureau of Refugees , Freedmen Abandoned Lands ...","['KELLEY', 'EDWARD McPHERSON', 'JONATHAN WORTH']","['Raleigh', 'N.C.', 'United States', 'Alabama'...","['Bureau of Refugees', 'House of Representativ...","['May 8th , 1866', 'May 1 , 1866', '1st']",[],"['2d', 'III']","['8-May-1866', '1-May-1866']","['8-May-1866', '1-May-1866']","['KELLEY', 'EDWARD McPHERSON', 'E. WHITTLESEY'...","['State of North Carolina', 'Raleigh', 'N.C.',...","['Bureau of Refugees , Freedmen Abandoned Land...",[],['JONATHAN WORTH'],"['Raleigh', 'N.C.', 'United States', 'Alabama'...","['Officers Agents of Bureau', 'House of Repres...",[]
3,"HEADQUARTERS DEPARTMENT OF NORTH CAROLINA , Ra...",['Joseph E. Johnston'],"['Raleigh', 'North Carolina']","['HEADQUARTERS DEPARTMENT OF NORTH CAROLINA', ...","['January 12th , 1866']",[],"['4', 'II', 'III']",['12-January-1866'],['12-January-1866'],['Joseph E. Johnston'],"['NORTH CAROLINA', 'Raleigh', 'North Carolina'...",['Government'],[],['Joseph E. Johnston'],"['NORTH CAROLINA', 'Raleigh', 'North Carolina']",[],[]
4,IV . Animals sold counties collected unless wo...,"['J A . Campbell', 'James H Anderson']",['U.S.'],"['Headquarters', 'Government']","['first day of February , proximo']",[],['VI'],[],[],"['J A . Campbell', 'James H Anderson']","['U.S.', 'U.S. C.S', 'C.S']",['Government'],[],['James H Anderson'],['U.S.'],[],['C.S']


In [15]:
order_text=order_ent["order_text"]
order_Per_onto=order_ent["PERSON-ontonotes"]
order_GPE_onto=order_ent["GPE-ontonotes"]
order_ORG_onto=order_ent["ORG-ontonotes"]
order_date_onto=order_ent["standard_date-ontonotes"]
order_loc_onto=order_ent["LOC-ontonotes"]

In [16]:
order_text[1000]

'190 By Order of Col. James V. Bomford Jacob F. Chur . Bvt . Lt. Col. A.A.A.G . Bureau of Refugees Freedmen A.L . Head Qrs . Asst . Commissioner Raleigh N.C. Jan. 16th 1867 Special Order No . 8 Leave of absence twelve days hereby granted Lieut Edgar H. McQuigg V.R.C . Asst Supt Bureau R.F . A.L . Wilmington N.C. By Command of Col. J. V. Bomford Jacob F. Chur . Bvt . Lt. Col. A.A.A . Genl Bureau of Refugees Freedmen c Hd Qrs Asst Commissioner Raleigh N.C. Jan. 18th 1867 Special Order No . 9 In compliance instructions Maj Genl O.O . Howard Commissioner , Bvt . Maj. Thos P. Johnston Chf Qr Mr Bureau State pay trustees of Baptist Female Seminary following amounts rent of rooms used Bureau upon premises known Guion Hotel City of Raleigh recommended board of Officers appointed purpose . For three rooms main building occupied offices of Bureau of R.F . A.L . April 1st 1866 Aug 1st 1866 rate of Twelve dollars per month per room . For four rooms detatched building occupied offices of Bureau of 

In [17]:
order_Per_onto[1000]

"['James V. Bomford', 'J. V. Bomford', 'Manchester W. Weld', 'Justin Hodge', 'M. W. Weld', 'Jacob F. Chur', 'F . A. Seeley']"

In [18]:
order_GPE_onto[1000]

"['Raleigh N.C', 'Raleigh', 'Fayetteville N.C.']"

In [19]:
order_ORG_onto[1000]

"['Bureau of Refugees Freedmen A.L', 'Bureau of Refugees Freedmen', 'Baptist Female Seminary', 'Bureau of R.F . A.L']"

In [20]:
order_loc_onto[1000]

'[]'

In [21]:
order_date_onto[1000]

"['16-Jan.-1867', '18-Jan.-1867', '1-April-1866', '1-August-1866', '2-April-1866', '31-December-1866', '19-Jan.-1867', '20-Jan.-1867']"

**good**

In [22]:
PL=[]
empty=0
for i in order_Per_onto :
    for j in i:
        PL.append(j)
for i in order_Per_onto:
    if i==[]:
        empty=empty+1
len(PL), empty

(453588, 0)

**very good**

In [23]:
PL=[]
empty=0
for i in order_date_onto :
    for j in i:
        PL.append(j)
for i in order_date_onto:
    if i=="[]":
        empty=empty+1
len(PL), empty

(167185, 2515)

**uii very good**

In [24]:
PL=[]
empty=0
for i in order_GPE_onto :
    for j in i:
        PL.append(j)
for i in order_GPE_onto:
    if i=="[]":
        empty=empty+1
len(PL), empty

(343109, 1295)

****

# Order Ner-english

In [25]:
order_ent.head()

,order_text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,postDate-ontonotes,standard_date-ontonotes,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Other_name-nerenglish
0,R G. R.R . THIS ORDER NOT TRANSFERABLE . U.S. ...,"['E . Cranford', 'Lewis B. Parsons', 'Major Kn...","['Raleigh N.C', 'Warrenton N.C', 'New York']",['Samuel Gale Co .'],"['Aug 6 1866', 'Aug 1866']",[],['67487'],['6-Aug-1866'],['6-August-1866'],"['Cranford', 'Lewis B. Parsons', 'Gaston R.R']","['U.S.', 'Raleigh N.C', 'Warrenton N.C', 'Rale...","['Samuel Gale Co', 'Regiment Freedman', 'Rail ...",[],[],"['Ral', 'Gaston R.R', 'Raleigh N.C.', 'Broadwa...",['Cranford'],[]
1,"Certified Copy of Order , Extract , within tra...",['E. Crawford'],['Raleigh N.C'],[],['Aug 6 1866'],[],[],['6-Aug-1866'],['6-August-1866'],"['Robinson Clinton', 'E. Crawford']",['Raleigh N.C'],[],[],[],[],[],[]
2,"Bureau of Refugees , Freedmen Abandoned Lands ...","['KELLEY', 'EDWARD McPHERSON', 'JONATHAN WORTH']","['Raleigh', 'N.C.', 'United States', 'Alabama'...","['Bureau of Refugees', 'House of Representativ...","['May 8th , 1866', 'May 1 , 1866', '1st']",[],"['2d', 'III']","['8-May-1866', '1-May-1866']","['8-May-1866', '1-May-1866']","['KELLEY', 'EDWARD McPHERSON', 'E. WHITTLESEY'...","['State of North Carolina', 'Raleigh', 'N.C.',...","['Bureau of Refugees , Freedmen Abandoned Land...",[],['JONATHAN WORTH'],"['Raleigh', 'N.C.', 'United States', 'Alabama'...","['Officers Agents of Bureau', 'House of Repres...",[]
3,"HEADQUARTERS DEPARTMENT OF NORTH CAROLINA , Ra...",['Joseph E. Johnston'],"['Raleigh', 'North Carolina']","['HEADQUARTERS DEPARTMENT OF NORTH CAROLINA', ...","['January 12th , 1866']",[],"['4', 'II', 'III']",['12-January-1866'],['12-January-1866'],['Joseph E. Johnston'],"['NORTH CAROLINA', 'Raleigh', 'North Carolina'...",['Government'],[],['Joseph E. Johnston'],"['NORTH CAROLINA', 'Raleigh', 'North Carolina']",[],[]
4,IV . Animals sold counties collected unless wo...,"['J A . Campbell', 'James H Anderson']",['U.S.'],"['Headquarters', 'Government']","['first day of February , proximo']",[],['VI'],[],[],"['J A . Campbell', 'James H Anderson']","['U.S.', 'U.S. C.S', 'C.S']",['Government'],[],['James H Anderson'],['U.S.'],[],['C.S']


In [26]:
order_Per_ner=order_ent["PERSON-nerenglish"]
order_ORG_ner=order_ent["ORG-nerenglish"]
order_other_ner=order_ent["Other_name-nerenglish"]
order_loc_ner=order_ent["LOC-nerenglish"]

In [27]:
order_text[2]

'Bureau of Refugees , Freedmen Abandoned Lands , Headquarters Assistant Commissioner , State of North Carolina , Raleigh , N.C. , May 8th , 1866 . Circular No . 4 The attention of Officers Agents of Bureau called following resolution of House of Representatives : THIRTYNINTH CONGRESSFIRST SESSION . Congress of United States . In House of Representatives , May 1 , 1866 . Mr. KELLEY submitted following , adopted : WHEREAS , It reported citizens of Alabama , formal memorial two Houses of Congress , many of people of mountain districts of State suffering want of adequate supplies of food , considerable numbers of died of actual starvation ; therefore , Be resolved , That President requested instruct proper officers of Bureau of Refugees Freedman enquire condition of said districts , districts of late insurgent States , suffering may said exist , relieve people thereof , provide corn seed planting crop sufficient annual supply of family requiring relief . ATTEST : Signed EDWARD McPHERSON , 

In [28]:
order_Per_ner[2]

"['JONATHAN WORTH']"

In [29]:
order_ORG_ner[2]

"['Officers Agents of Bureau', 'House of Representatives', 'Houses of Congress', 'Bureau of Refugees Freedman', 'Freedmens Bureau']"

**names of ner english are catastrophic**

****

# Order Ner english large 

In [30]:
order_Per_large=order_ent["PERSON-nerenglarge"]
order_ORG_large=order_ent["ORG-nerenglarge"]
order_other_large=order_ent["other_namelarge"]
order_loc_large=order_ent["LOC-nerenglarge"]

In [31]:
order_text[1000]

'190 By Order of Col. James V. Bomford Jacob F. Chur . Bvt . Lt. Col. A.A.A.G . Bureau of Refugees Freedmen A.L . Head Qrs . Asst . Commissioner Raleigh N.C. Jan. 16th 1867 Special Order No . 8 Leave of absence twelve days hereby granted Lieut Edgar H. McQuigg V.R.C . Asst Supt Bureau R.F . A.L . Wilmington N.C. By Command of Col. J. V. Bomford Jacob F. Chur . Bvt . Lt. Col. A.A.A . Genl Bureau of Refugees Freedmen c Hd Qrs Asst Commissioner Raleigh N.C. Jan. 18th 1867 Special Order No . 9 In compliance instructions Maj Genl O.O . Howard Commissioner , Bvt . Maj. Thos P. Johnston Chf Qr Mr Bureau State pay trustees of Baptist Female Seminary following amounts rent of rooms used Bureau upon premises known Guion Hotel City of Raleigh recommended board of Officers appointed purpose . For three rooms main building occupied offices of Bureau of R.F . A.L . April 1st 1866 Aug 1st 1866 rate of Twelve dollars per month per room . For four rooms detatched building occupied offices of Bureau of 

In [32]:
order_Per_large[1000]

"['James V. Bomford', 'Jacob F. Chur', 'Edgar H .', 'McQuigg', 'J. V. Bomford', 'Genl O.O . Howard', 'Thos P. Johnston', 'Manchester W. Weld', 'Justin Hodge', 'M. W. Weld', 'F . A. Seeley']"

In [33]:
order_ORG_large[1000]

"['Bureau of Refugees Freedmen A.L', 'Bureau R.F . A.L', 'Bureau of Refugees Freedmen', 'Baptist Female Seminary', 'Bureau', 'Guion Hotel', 'Bureau of R.F . A.L', 'Bureau of R.F . A.L .']"

In [34]:
order_loc_large[1000]

"['Raleigh N.C', 'Wilmington N.C', 'City of Raleigh', 'Fayetteville N.C.']"

In [35]:
order_other_large[102]

"['General Orders']"

**veryy good**

In [36]:
PL=[]
empty=0
for i in order_Per_large :
    for j in i:
        PL.append(j)
for i in order_Per_large:
    if i=='[]':
        empty=empty+1
len(PL), empty

(556132, 1438)

In [37]:
PL=[]
empty=0
for i in order_ORG_large :
    for j in i:
        PL.append(j)
for i in order_ORG_large:
    if i=='[]':
        empty=empty+1
len(PL), empty

(564296, 1382)

In [38]:
PL=[]
empty=0
for i in order_loc_large :
    for j in i:
        PL.append(j)
for i in order_loc_large:
    if i=='[]':
        empty=empty+1
len(PL), empty

(543223, 858)

# Ner enlish large is better than ner english in extracted entities (as usuall i think)

****

# Contracts Ontonotes

In [39]:
contract_ent=pd.read_csv("../input/all-csv/to_compare_result_contract_ner.csv")
contract_onto=pd.read_csv("../input/all-csv/contract_onto (1).csv")

In [40]:
contract_ent.head()

,contract_text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Name-nerenglish,Other_name-nerenglish,Name-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge
0,North Carolina This indenture made entered Thi...,"['Thos R Underwood', 'Alfred A McKerhan', 'Wm ...","['County of Carteret', 'Shepperd Creek', 'Cree...",[],"['Thos R Underwood', 'Alfred A McKerhan', 'Wm ...",[],"['H.G . Cutler', 'Thos R Underwood', 'Alfred A...","['North Carolina', 'County of Carteret', 'Bogu...",[],[]
1,State of North Carolina Currituck County . Thi...,['John B. Etheridge'],"['County State', 'Witnesseth', 'County of Curr...",[],['John B. Etheridge'],[],"['John B. Etheridge', 'Little John Etheridge']","['State of North Carolina', 'Currituck County'...",[],[]
2,"near Lizzie Etheridges gate , thence South 56 ...","['John B Etheridge', 'Douglas Adam D. Etheridge']","['Red Hill', 'Croatan Sound']",[],"['John B Etheridge', 'Douglas Adam D. Etheridge']",[],"['Little John Etheridge', 'John B Etheridge', ...","['Lizzie Etheridges gate', 'Red Hill', 'Croata...",[],[]
3,Copy Agreement of Apprenticeship Bu Mrs. Kate ...,['Kate V.Chamblin'],[],[],['Kate V.Chamblin'],[],"['Kate V.Chamblin', '1st', 'Geo . W. Rollins',...","['La', 'Lake Providence Carroll']",['U.S. Army'],[]
4,Copy Agreement of Apprenticeship . This agreem...,"['Kate V. Chambliss', 'Betty Taylor Mary Taylo...",[],[],"['Kate V. Chambliss', 'Betty Taylor Mary Taylo...",['African'],"['Mrs', 'Kate V. Chambliss', 'James Samenett',...","['Carroll Parish', 'New Orleans', 'Louisiana',...",[],['African']


In [41]:
contract_onto.head()

,contract_text,Name-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,Cardinal-ontonotes,postDate-ontonotes,standard_date-ontonotes
0,North Carolina This indenture made entered Thi...,"['H.G . Cutler', 'Thos R Underwood', 'Alfred A...","['North Carolina', 'County of Carteret']",[],['This 5th day of November 1853'],"['Stake North', 'Bogue Sound', 'Shepperd Creek']",[],"['five hundred ten', '13.0', '203', '209', '4'...","['This 5-November-1853', '5-This-1853', 'This-...","['5-November-1853', '5-This-1853', '7-November..."
1,State of North Carolina Currituck County . Thi...,"['John B. Etheridge', 'Little John Etheridge',...","['North Carolina', 'Currituck County', 'County...",[],"['fifth day of September , year', 'One Thousan...","['Roanoke Island', 'Croatain Sound']",[],[],[],[]
2,"near Lizzie Etheridges gate , thence South 56 ...","['Lizzie Etheridges', 'Little John Etheridge',...",[],[],[],"['Red Hill', 'Croatan Sound', 'Sound']",[],[],[],[]
3,Copy Agreement of Apprenticeship Bu Mrs. Kate ...,"['Kate V.Chamblin', 'James Samenett', 'Betty T...",['Carroll Parish La'],['U.S. Army'],"['Oct. 1 1866', 'Nov. 3 1866']",[],[],[],"['1-Oct.-1866', '3-Nov.-1866']","['1-Oct.-1866', '3-Nov.-1866']"
4,Copy Agreement of Apprenticeship . This agreem...,"['Kate V. Chambliss', 'James Samenett', 'Betty...",['New Orleans Louisiana'],[],"['1 day of October A . D. 1866', 'Octr 31 1865...",[],[],['two'],"['1-October-1866', '31-Octr-1865', '1-Oct.-186...","['1-October-1866', '31-Octr-1865', '1-Oct.-186..."


In [42]:
contract_text=contract_ent["contract_text"]
contract_Per_onto=contract_onto["Name-ontonotes"]
contract_GPE_onto=contract_onto["GPE-ontonotes"]
contract_ORG_onto=contract_onto["ORG-ontonotes"]
contract_date_onto=contract_onto["standard_date-ontonotes"]
contract_loc_onto=contract_onto["LOC-ontonotes"]
contract_cardinal=contract_onto["Cardinal-ontonotes"]

In [43]:
contract_text[0]

'North Carolina This indenture made entered This 5th day of November 1853 . By H.G . Cutler of County of Carteret of first part Thos R Underwood , Alfred A McKerhan , Wm H Washington , Geo . S Stevenson , John D Williams Nathan A Stedman of second part . Whereas parties of Second part divers others associates purchased of said H. G. Cutler , tract of land situated County of Carteret bounded described follows ; beginning Stake North side of Bogue Sound running thence westwardly various corners of sound , five hundred ten poles Straight . Thence N. 13.0 E. 203 poles , thence East 209 poles , south 4 poles , thence North 85 , E. 116 poles , thence W. 20.0 W. 96 poles , thence N 45.0 E. 21 12 poles thence E. 29 12 poles , thence S. 83.0 E. 220 poles , thence South 5.0 W. Shepperd Creek thence Westerly said Creek South West Corner of tract of land converged said Cutler John Sabuton Then S 50 West along Sabutons West line beginning , containing Estimation Nine hundred four acres survey of sa

In [44]:
contract_date_onto[150]

'[]'

In [45]:
contract_Per_onto[0]

"['H.G . Cutler', 'Thos R Underwood', 'Alfred A McKerhan', 'John D Williams', 'Nathan A Stedman', 'H . G. Cutler', 'Cutler', 'William H Marshall']"

In [46]:
contract_GPE_onto[0]

"['North Carolina', 'County of Carteret']"

In [47]:
contract_ORG_onto[0]

'[]'

In [48]:
contract_loc_onto[0]

"['Stake North', 'Bogue Sound', 'Shepperd Creek']"

In [49]:
contract_cardinal[0]

"['five hundred ten', '13.0', '203', '209', '4', '85', '116', '20.0', '96', '45.0', '21 12', '29 12', '83.0', '220', '5.0', '50']"

In [50]:
PL=[]
empty=0
for i in contract_date_onto :
    for j in i:
        PL.append(j)
for i in contract_date_onto:
    if i=='[]':
        empty=empty+1
len(PL), empty

(426318, 10160)

In [51]:
PL=[]
empty=0
for i in contract_Per_onto :
    for j in i:
        PL.append(j)
for i in contract_Per_onto:
    if i=='[]':
        empty=empty+1
len(PL), empty

(1525017, 3304)

In [52]:
PL=[]
empty=0
for i in contract_GPE_onto :
    for j in i:
        PL.append(j)
for i in contract_GPE_onto:
    if i=='[]':
        empty=empty+1
len(PL), empty

(358023, 12932)

In [53]:
PL=[]
empty=0
for i in contract_loc_onto :
    for j in i:
        PL.append(j)
for i in contract_loc_onto:
    if i=='[]':
        empty=empty+1
len(PL), empty

(60156, 23015)

****

# Contracts Ner english

In [54]:
contract_ent.head()

,contract_text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Name-nerenglish,Other_name-nerenglish,Name-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge
0,North Carolina This indenture made entered Thi...,"['Thos R Underwood', 'Alfred A McKerhan', 'Wm ...","['County of Carteret', 'Shepperd Creek', 'Cree...",[],"['Thos R Underwood', 'Alfred A McKerhan', 'Wm ...",[],"['H.G . Cutler', 'Thos R Underwood', 'Alfred A...","['North Carolina', 'County of Carteret', 'Bogu...",[],[]
1,State of North Carolina Currituck County . Thi...,['John B. Etheridge'],"['County State', 'Witnesseth', 'County of Curr...",[],['John B. Etheridge'],[],"['John B. Etheridge', 'Little John Etheridge']","['State of North Carolina', 'Currituck County'...",[],[]
2,"near Lizzie Etheridges gate , thence South 56 ...","['John B Etheridge', 'Douglas Adam D. Etheridge']","['Red Hill', 'Croatan Sound']",[],"['John B Etheridge', 'Douglas Adam D. Etheridge']",[],"['Little John Etheridge', 'John B Etheridge', ...","['Lizzie Etheridges gate', 'Red Hill', 'Croata...",[],[]
3,Copy Agreement of Apprenticeship Bu Mrs. Kate ...,['Kate V.Chamblin'],[],[],['Kate V.Chamblin'],[],"['Kate V.Chamblin', '1st', 'Geo . W. Rollins',...","['La', 'Lake Providence Carroll']",['U.S. Army'],[]
4,Copy Agreement of Apprenticeship . This agreem...,"['Kate V. Chambliss', 'Betty Taylor Mary Taylo...",[],[],"['Kate V. Chambliss', 'Betty Taylor Mary Taylo...",['African'],"['Mrs', 'Kate V. Chambliss', 'James Samenett',...","['Carroll Parish', 'New Orleans', 'Louisiana',...",[],['African']


In [55]:
contract_Per_ner=contract_ent["PERSON-nerenglish"]
contract_loc_ner=contract_ent["LOC-nerenglish"]
contract_ORG_ner=contract_ent["ORG-nerenglish"]
contract_other_ner=contract_ent["Other_name-nerenglish"]

In [56]:
contract_text[1000]

'AGREEMENT FOR LABOR . THIS INDENTURE , Made entered day of A.D , 186 , of County of State of North Carolina , of first part , undersigned , Free Laborer , of second part : WITNESSETH , That said Laborers agreed work said date day of 186 186 ; kinds of work common farms of country ; work faithfully , respectful deportment . And agree , receive every month , amount placed opposite names , rest end of year ; amount retained shall forfeited violation of contract clearly proven . And agree , time lost idleness absence without leave , shall paid . And said agreed furnish said laborers comfortable quarters , sufficient rations , amount of money per month , stands opposite respective names ; treat kindly , encourage establishment of schools children . Henry Pearce No . NAME . Age . Wages . DOLLS . CTS . Monthly Paymt . DOLLS . CTS . Henry Pearce Tho H Hay Assist . Supt . Bureau R. F. A. L. , SubDistrict of'

In [57]:
contract_Per_ner[1000]

"['Henry Pearce']"

In [58]:
contract_loc_ner[1000]

'[]'

In [59]:
contract_ORG_ner[1000]

"['Henry Pearce Tho H Hay Assist']"

In [60]:
contract_other_ner[1000]

'[]'

**not bad but ontonotes still better**

In [61]:
PL=[]
empty=0
for i in contract_Per_ner :
    for j in i:
        PL.append(j)
for i in contract_Per_ner:
    if i=='[]':
        empty=empty+1
len(PL), empty

(790998, 7423)

****

In [62]:
contract_ent.head()

,contract_text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Name-nerenglish,Other_name-nerenglish,Name-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge
0,North Carolina This indenture made entered Thi...,"['Thos R Underwood', 'Alfred A McKerhan', 'Wm ...","['County of Carteret', 'Shepperd Creek', 'Cree...",[],"['Thos R Underwood', 'Alfred A McKerhan', 'Wm ...",[],"['H.G . Cutler', 'Thos R Underwood', 'Alfred A...","['North Carolina', 'County of Carteret', 'Bogu...",[],[]
1,State of North Carolina Currituck County . Thi...,['John B. Etheridge'],"['County State', 'Witnesseth', 'County of Curr...",[],['John B. Etheridge'],[],"['John B. Etheridge', 'Little John Etheridge']","['State of North Carolina', 'Currituck County'...",[],[]
2,"near Lizzie Etheridges gate , thence South 56 ...","['John B Etheridge', 'Douglas Adam D. Etheridge']","['Red Hill', 'Croatan Sound']",[],"['John B Etheridge', 'Douglas Adam D. Etheridge']",[],"['Little John Etheridge', 'John B Etheridge', ...","['Lizzie Etheridges gate', 'Red Hill', 'Croata...",[],[]
3,Copy Agreement of Apprenticeship Bu Mrs. Kate ...,['Kate V.Chamblin'],[],[],['Kate V.Chamblin'],[],"['Kate V.Chamblin', '1st', 'Geo . W. Rollins',...","['La', 'Lake Providence Carroll']",['U.S. Army'],[]
4,Copy Agreement of Apprenticeship . This agreem...,"['Kate V. Chambliss', 'Betty Taylor Mary Taylo...",[],[],"['Kate V. Chambliss', 'Betty Taylor Mary Taylo...",['African'],"['Mrs', 'Kate V. Chambliss', 'James Samenett',...","['Carroll Parish', 'New Orleans', 'Louisiana',...",[],['African']


In [63]:
contract_Per_large=contract_ent["Name-nerenglarge"]
contract_loc_large=contract_ent["LOC-nerenglarge"]
contract_ORG_large=contract_ent["ORG-nerenglarge"]
contract_other_large=contract_ent["other_namelarge"]

In [64]:
contract_text[0]

'North Carolina This indenture made entered This 5th day of November 1853 . By H.G . Cutler of County of Carteret of first part Thos R Underwood , Alfred A McKerhan , Wm H Washington , Geo . S Stevenson , John D Williams Nathan A Stedman of second part . Whereas parties of Second part divers others associates purchased of said H. G. Cutler , tract of land situated County of Carteret bounded described follows ; beginning Stake North side of Bogue Sound running thence westwardly various corners of sound , five hundred ten poles Straight . Thence N. 13.0 E. 203 poles , thence East 209 poles , south 4 poles , thence North 85 , E. 116 poles , thence W. 20.0 W. 96 poles , thence N 45.0 E. 21 12 poles thence E. 29 12 poles , thence S. 83.0 E. 220 poles , thence South 5.0 W. Shepperd Creek thence Westerly said Creek South West Corner of tract of land converged said Cutler John Sabuton Then S 50 West along Sabutons West line beginning , containing Estimation Nine hundred four acres survey of sa

In [65]:
contract_Per_large[0]

"['H.G . Cutler', 'Thos R Underwood', 'Alfred A McKerhan', 'Wm H Washington', 'Geo . S Stevenson', 'John D Williams', 'Nathan A Stedman', 'H . G. Cutler', 'Cutler John Sabuton', 'William H Marshall']"

In [66]:
contract_loc_large[0]

"['North Carolina', 'County of Carteret', 'Bogue Sound', 'Shepperd Creek', 'Westerly', 'Sabutons West']"

In [67]:
contract_ORG_large[0]

'[]'

In [68]:
contract_other_large[0]

'[]'

In [69]:
PL=[]
empty=0
for i in contract_Per_large :
    for j in i:
        PL.append(j)
for i in contract_Per_large:
    if i=='[]':
        empty=empty+1
len(PL), empty

(1717997, 3532)

# the better one as usuall 

****

# Records Financial

In [70]:
recordF_ent=pd.read_csv("../input/all-csv/to_compare_result_recordF.csv")

In [71]:
recordF_ent.head(1)

,recordF_text,Name-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,Cardinal-ontonotes,postDate-ontonotes,standard_date-ontonotes,Name-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Other_name-nerenglish
0,Quarter ending March 31st 1866 Justin Hodge Ca...,"['Justin Hodge', 'Henry Swim', 'JC Barnett', '...","['Nottsburg', 'Salisbury']",['US Treasury'],"['Quarter ending March 31st 1866', 'Dec 18', '...",[],[],['One'],"['31-Quarter ending March-1866', 'Quarter-Marc...",['Quarter-March-1866'],"['Justin Hodge', 'Henry Swim', 'Wm B Meins', '...","['Dist West', 'N.Ca', 'Greensboro N.C', 'Notts...","['A.Q.M . Financial Agent Bureau', 'Fines', 'A...",[],"['Jacob Bodecker', 'Justin Hodge']",[],[],[]


In [72]:
recordF_text=recordF_ent["recordF_text"]
recordF_Per_onto=recordF_ent["Name-ontonotes"]
recordF_GPE_onto=recordF_ent["GPE-ontonotes"]
recordF_ORG_onto=recordF_ent["ORG-ontonotes"]
recordF_date_onto=recordF_ent["standard_date-ontonotes"]
recordF_loc_onto=recordF_ent["LOC-ontonotes"]
recordF_cardinal=recordF_ent["Cardinal-ontonotes"]

In [73]:
recordF_text[300]

'Hugh To amount brought forward 60.96 May 19 1 mug 25.2 mackerel 15 1 qt . Brandy 1.35 1.80 May 21 3 lbs Copperas 10 Box blacking 10 .40 May 22 1 Doz Herring 50 . 23rd . 1 12 qts . brandy 1.25 2.37 May 26 12 doz . Herring 50 Jun 23rd Brandy 1.3 .88 June 3 2 cotton Hdkfs .60 1.20 June 9 2 Home made Cotton Shirts 2.00 4.00 June 2 pr . Country made pants 2.50 5.00 June 1 12 doz . Buttons 10 1 qt Brandy 1.25 1.40 June 15 1 cups saucers 1.25 1.25 June 16 1 pr . Cotton Stockings 75 75 June 21 1 lb Sugar 18 2 lb Adam T. Candles 50 43 June 22 12 Gallon Brandy 5.00 2.50 June 1 Cotton Hdkfs 40 40 June 25 1 Pocket Knife 6 2 Spools thread 15 1.30 June 1 yd . Calico 25 25 June 29 1 quart Whiskey 1.25 1.25 July 7 12 Gallon Do 5.00 2.50 July 7 1 Pad Lock 37 2 1 lb . Sugar 18 55 July 13 12 lb . Adm. Candles 50 1 lb . Nails 10 35 July 1 Box Matches 5 25 lb . 2 qts . Whiskey 1.25 2.55 July 28 1 quart Brandy 1.25 1.25 Augt . 3 3¼ yds . Capeniars 1.50 4.87 Augt . 3 1 Bunch hemp Thread 10 1 doz P. Buttons 

In [74]:
recordF_cardinal[300]

"['60.96', '25.2', '15', '1.35', '10', '40', '50', '12', '1.25', '1.3', '88', '3', '2', '1.25 1.25', '18', '6', '1.30', '37', '35', '1', '5', '1.50', '20', '63', '30', '2.00', '4.50', '62', '14', '25', '2.81', '60', '11.00', '60.17', '3.85', '1.60', '70', '74.20', '135.16', '2.50', '42.50', '½', '1.20', '44.74', '102.44', '37.22']"

In [75]:
recordF_date_onto[300]

"['25-February-1868']"

In [76]:
recordF_Per_onto[300]

"['Hugh', 'Stephen Neal', 'Fariss', 'G J Farish', 'Nelson Dortgito', 'G . J. Farish']"

In [77]:
recordF_GPE_onto[300]

'[]'

In [78]:
recordF_ORG_onto[300]

'[]'

In [79]:
PL=[]
empty=0
for i in recordF_date_onto :
    for j in i:
        PL.append(j)
for i in recordF_date_onto:
    if i=='[]':
        empty=empty+1
len(PL), empty

(50173, 1769)

In [80]:
PL=[]
empty=0
for i in recordF_Per_onto :
    for j in i:
        PL.append(j)
for i in recordF_Per_onto:
    if i=='[]':
        empty=empty+1
len(PL), empty

(115920, 925)

****

In [81]:
recordE_ent=pd.read_csv("../input/all-csv/to_compare_result_recordE.csv")
recordE_text=recordE_ent["recordE_text"]
recordE_Per_onto=recordE_ent["Name-ontonotes"]
recordE_GPE_onto=recordE_ent["GPE-ontonotes"]
recordE_ORG_onto=recordE_ent["ORG-ontonotes"]
recordE_date_onto=recordE_ent["standard_date-ontonotes"]
recordE_loc_onto=recordE_ent["LOC-ontonotes"]
recordE_cardinal=recordE_ent["Cardinal-ontonotes"]

In [82]:
recordE_text[45]

'28 Spanish Neck Co. Edesville Md . School House 1867 Mar . 1 From Barracks , Stables c. Balto Asson . 12,224 feet Lumber 11 Windows 3 Doors 15 Sec . of Roof 205.48 29 Talbot Co. Trappe Md . School House 1867 Mar . 1 From Barracks , Stables c. Balto Asson . 10,192 feet Lumber 10 Windows 2 Doors 12 Sec . of Roof 163.20'

In [83]:
recordE_ORG_onto[45]

"['Spanish Neck Co .', 'School House', 'Talbot Co .']"

In [84]:
recordE_loc_onto[45]

'[]'

In [85]:
recordE_date_onto[45]

'[]'

In [86]:
recordE_GPE_onto[45]

"['Edesville', 'Md', 'Trappe']"

In [87]:
recordE_Per_onto[1000]

"['J . T Kirkman', 'Hartnett', 'D F Allen']"

In [88]:
PL=[]
empty=0
for i in recordE_Per_onto :
    for j in i:
        PL.append(j)
for i in recordE_Per_onto:
    if i=='[]':
        empty=empty+1
len(PL), empty

(148161, 2868)

In [89]:
PL=[]
empty=0
for i in recordE_date_onto :
    for j in i:
        PL.append(j)
for i in recordE_date_onto:
    if i=='[]':
        empty=empty+1
len(PL), empty

(73441, 4916)

In [90]:
PL=[]
empty=0
for i in recordE_ORG_onto :
    for j in i:
        PL.append(j)
for i in recordE_ORG_onto :
    if i=='[]':
        empty=empty+1
len(PL), empty

(95379, 5150)

****

In [91]:
recordC_ent=pd.read_csv("../input/all-csv/to_compare_result_recordC.csv")
recordC_text=recordC_ent["recordC_text"]
recordC_Per_onto=recordC_ent["Name-ontonotes"]
recordC_GPE_onto=recordC_ent["GPE-ontonotes"]
recordC_ORG_onto=recordC_ent["ORG-ontonotes"]
recordC_date_onto=recordC_ent["standard_date-ontonotes"]
recordC_loc_onto=recordC_ent["LOC-ontonotes"]

In [92]:
recordC_text[0]

'one hundred dollars . I never heard Bagby complain heard say dared Mr Richardson whip W.M . Henry Signed sealed sworn , March 19th 1866 . H.H . Holton J . Peace Edward Homer sworn saith never saw knew of Mr Richardsons whipping correcting said Bagby , good reason prudence demanded chastisement , proper manner , never bodily injured , caused ly stop labor , eating , correction Mr Edward Horner seal Sworn subscribed March 19 1866 H.H . Holton Examines Court Mathew Baldwin sworn saith heard Bagly speak of contract Mr Richardson expressed satisfied I advised good trade contract , never complained of treatment , said well treated When asked anything got , goes well dressed frequently comes perhaps goes anywhere else . I know of one better treated , Mathew x mark Baldwin H.H . Holton'

In [93]:
recordC_loc_onto[0]

'[]'

In [94]:
recordC_date_onto[0]

"['19-March-1866', '19-March-1866']"

In [95]:
recordC_GPE_onto[0]

'[]'

In [96]:
recordC_ORG_onto[0]

'[]'

In [97]:
recordC_Per_onto[0]

"['Bagby', 'Richardson', 'W.M . Henry', 'H.H . Holton', 'Edward Homer', 'Richardsons', 'Edward Horner', 'Mathew Baldwin', 'Bagly', 'Mathew']"